In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# 제품 데이터 저장 리스트
products_data = []

# Selenium 드라이버 설정
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    # Selenium 탐지 우회
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 제품 정보 추출 함수
def extract_product_info(product, driver):
    try:
        if product.get("id", "").startswith("adPointProductItem") or "adSmartAreaTop" in product.get("id", ""):
            return False

        photo_elem = product.select_one(".thumb_image img")
        if photo_elem:
            photo = photo_elem.get("data-original") or photo_elem.get("src") or "N/A"
            if "noImg_130.gif" in photo:
                photo = "N/A"
        else:
            photo = "N/A"

        name_elem = product.select_one(".prod_name a")
        name = name_elem.text.strip() if name_elem else "Unknown"
        detail_url = name_elem["href"] if name_elem and "href" in name_elem.attrs else None

        # 상세 페이지에서 spec 수집
        spec_dict = {}
        if detail_url:
            # 현재 탭의 핸들 저장
            main_window = driver.current_window_handle

            # 새 탭 열기 (JavaScript 실행)
            driver.execute_script("window.open('', '_blank');")

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 새 탭에서 상세 페이지 열기
            driver.get(detail_url)
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_spec table"))
                )
            except:
                print(f"❗️스펙 테이블 로딩 실패 (대기 시간 초과): {name}")

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            spec_table = detail_soup.select_one(".prod_spec .spec_tbl tbody") or detail_soup.select_one(".prod_spec table")
            if spec_table:
                for row in spec_table.find_all("tr"):
                    keys = row.find_all("th", class_="tit")
                    values = row.find_all("td", class_="dsc")
                    for key_elem, value_elem in zip(keys, values):
                        key = key_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)
                        spec_dict[key] = value

            # 탭 닫기
            driver.close()

            # 원래 탭으로 돌아가기
            driver.switch_to.window(main_window)

        price_elem = product.select_one(".price_sect a")
        price = price_elem.text.strip().replace(",", "") if price_elem else "N/A"

        rating_elem = product.select_one(".star-single .text__score") or product.select_one(".point_num")
        rating = rating_elem.text.strip() if rating_elem else "N/A"

        product_info = {
            "url": photo,
            "Name": name,
            "Price": price,
            "Rating": rating,
            **spec_dict
        }
        products_data.append(product_info)
        print(f"수집: {name}, 사진: {photo}, 가격: {price}, 평점: {rating}")
        return True

    except Exception as e:
        print(f"제품 정보 수집 오류: {e}")

        # 혹시 새 탭이 열려있는 상태라면 닫고 메인 탭으로 돌아가기
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass

        return False

# 페이지 스크레이핑 함수
def scrape_page(driver, page_num):
    try:
        # 현재 페이지의 제품 정보 수집
        soup = BeautifulSoup(driver.page_source, "html.parser")
        products = soup.select(".prod_item.prod_layer")
        print(f"페이지 {page_num}에서 찾은 제품 수: {len(products)}")

        if not products:
            print("더 이상 제품이 없습니다.")
            return False

        for product in products:
            if product.find_parent(id="adPointArea") or product.find_parent(id="adSmartAreaTop"):
                continue
            extract_product_info(product, driver)

        return True
    except Exception as e:
        print(f"페이지 수집 오류 (페이지 {page_num}): {e}")
        return False

# 크롤링 함수
def crawl_danawa_cpu_to_dataframe(max_pages=None):
    driver = setup_driver()
    try:
        base_url = "https://prod.danawa.com/list/?cate=112760"
        driver.get(base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기

        page_num = 1
        while True:
            if max_pages and page_num > max_pages:
                break

            print(f"페이지 {page_num} 수집 중...")

            # 현재 페이지 수집
            scrape_page(driver, page_num)

            if max_pages and page_num >= max_pages:
                break

            # 다음 페이지로 이동
            try:
                # 페이지 이동 방식 결정
                if page_num % 10 == 0:  # 10, 20, 30 등 10의 배수 페이지일 경우
                    # 다음 페이지 그룹 버튼 클릭
                    next_page_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.edge_nav.nav_next"))
                    )
                    next_page_button.click()
                else:
                    # 일반 페이지 이동
                    driver.execute_script(f"movePage({page_num + 1}); return false;")

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_item.prod_layer"))
                )
                time.sleep(2)  # 추가 대기 시간

                page_num += 1
            except Exception as e:
                print(f"다음 페이지로 이동 실패: {e}")
                break

        df = pd.DataFrame(products_data)
        print(f"총 {len(products_data)}개 CPU 제품 수집 완료")
        return df

    finally:
        driver.quit()
        print("Selenium 드라이버 종료")

# 실행
max_pages = 50  # 테스트를 위해 작은 값으로 설정
df = crawl_danawa_cpu_to_dataframe(max_pages=max_pages)
print("\n수집된 데이터프레임 미리보기:")
print(df.head())
df.to_csv("danawa_ssd_products.csv", index=False, encoding="utf-8-sig")
print("데이터가 'danawa_ssd_products.csv' 파일로 저장되었습니다.")

페이지 1 수집 중...
페이지 1에서 찾은 제품 수: 30
수집: SK하이닉스 Platinum P41 M.2 NVMe, 사진: //img.danawa.com/prod_img/500000/984/000/img/17000984_1.jpg?shrink=130:130&_v=20250116102807, 가격: 255000원, 평점: 4.9
수집: ESSENCORE KLEVV CRAS C910 M.2 NVMe, 사진: //img.danawa.com/prod_img/500000/154/849/img/58849154_1.jpg?shrink=130:130&_v=20250106082403, 가격: 153300원, 평점: 4.6
수집: 삼성전자 990 PRO M.2 NVMe, 사진: //img.danawa.com/prod_img/500000/722/297/img/18297722_1.jpg?shrink=130:130&_v=20240104172846, 가격: 500040원, 평점: 4.8
수집: Western Digital WD BLACK SN850X M.2 NVMe, 사진: //img.danawa.com/prod_img/500000/451/788/img/17788451_1.jpg?shrink=130:130&_v=20240403154623, 가격: 1006620원, 평점: 4.9
수집: SK하이닉스 Gold P31 M.2 NVMe, 사진: //img.danawa.com/prod_img/500000/338/161/img/13161338_1.jpg?shrink=130:130&_v=20231129150723, 가격: 225000원, 평점: 4.8
수집: 솔리다임 P44 Pro M.2 NVMe 벌크, 사진: //img.danawa.com/prod_img/500000/300/656/img/69656300_1.jpg?shrink=130:130&_v=20241023093417, 가격: 197000원, 평점: 4.9
수집: 마이크론 Crucial T500 M.2 NVMe 대원씨티에스, 사진: /

In [5]:
import pandas as pd
df = pd.read_csv("danawa_ssd_products.csv", encoding="UTF-8")
df

,url,Name,Price,Rating,제조회사,등록년월,제품분류,폼팩터,인터페이스,프로토콜,...,1년,M.2 (22110),기타,PLP,1.8형(4.6cm),2년,레드,10년,맥북 업그레이드용,오렌지
0,//img.danawa.com/prod_img/500000/984/000/img/1...,SK하이닉스 Platinum P41 M.2 NVMe,255000원,4.9,SK하이닉스(제조사 웹사이트 바로가기),2022년 05월,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,//img.danawa.com/prod_img/500000/154/849/img/5...,ESSENCORE KLEVV CRAS C910 M.2 NVMe,153300원,4.6,ESSENCORE(제조사 웹사이트 바로가기),2024년 07월,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,//img.danawa.com/prod_img/500000/722/297/img/1...,삼성전자 990 PRO M.2 NVMe,500040원,4.8,삼성전자(제조사 웹사이트 바로가기),2022년 11월,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,//img.danawa.com/prod_img/500000/451/788/img/1...,Western Digital WD BLACK SN850X M.2 NVMe,1006620원,4.9,Western Digital(제조사 웹사이트 바로가기),2022년 09월,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,//img.danawa.com/prod_img/500000/338/161/img/1...,SK하이닉스 Gold P31 M.2 NVMe,225000원,4.8,SK하이닉스(제조사 웹사이트 바로가기),2021년 01월,내장형SSD,M.2 (2280),PCIe3.0x4 (32GT/s),NVMe 1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,//img.danawa.com/prod_img/500000/194/958/img/1...,SOH SATA to USB3.0 어댑터(J-300),9800원,5.0,SOH,2013년 03월,SSD/HDD 주변기기,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,//img.danawa.com/prod_img/500000/117/649/img/1...,라이트컴 COMS A3992 하드케이스,1700원,4.2,라이트컴 COMS,2012년 04월,SSD/HDD 주변기기,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1361,//img.danawa.com/prod_img/500000/110/649/img/1...,라이트컴 COMS A3993 하드케이스,1700원,4.6,라이트컴 COMS,2012년 04월,SSD/HDD 주변기기,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1362,//img.danawa.com/prod_img/500000/346/297/img/1...,이지넷유비쿼터스 NEXT-HDD BOX 하드 보호케이스,3660원,4.7,이지넷유비쿼터스,2011년 01월,SSD/HDD 주변기기,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# c 컬럼의 인덱스 찾기
index = df.columns.get_loc('적합성평가인증')

# c 컬럼 좌측의 컬럼들 선택
left_columns = df.columns[:index]
df = df[left_columns]

# 공백을 기준으로 첫 번째 단어 추출
df['등록년'] = df['등록년월'].str.split().str[0]
df['등록월'] = df['등록년월'].str.split().str[1]
df['제조회사'] = df['제조회사'].str.split('\(').str[0]

df.drop(['등록년월', 'Unnamed: 25'], axis=1, inplace=True)

df

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
C:\Users\ms0\AppData\Local\Temp\ipykernel_12536\146649461.py:11: SyntaxWarning: invalid escape sequence '\('
  df['제조회사'] = df['제조회사'].str.split('\(').str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_12536\146649461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록년'] = df['등록년월'].str.split().str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_12536\146649461.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,url,Name,Price,Rating,제조회사,제품분류,폼팩터,인터페이스,프로토콜,용량,...,게임기 호환,5년,제한보증,NVMe 방열판,가로,세로,두께,무게,등록년,등록월
0,//img.danawa.com/prod_img/500000/984/000/img/1...,SK하이닉스 Platinum P41 M.2 NVMe,255000원,4.9,SK하이닉스,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,2TB,...,PS5 호환,○,○,방열판 미포함,80mm,22mm,2.38mm,7g,2022년,05월
1,//img.danawa.com/prod_img/500000/154/849/img/5...,ESSENCORE KLEVV CRAS C910 M.2 NVMe,153300원,4.6,ESSENCORE,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,1TB,...,NaN,○,○,방열판 포함,80mm,22mm,3.4mm,10g,2024년,07월
2,//img.danawa.com/prod_img/500000/722/297/img/1...,삼성전자 990 PRO M.2 NVMe,500040원,4.8,삼성전자,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 2.0,2TB,...,PS5 호환,○,○,방열판 미포함,80mm,22mm,2.3mm,9g,2022년,11월
3,//img.danawa.com/prod_img/500000/451/788/img/1...,Western Digital WD BLACK SN850X M.2 NVMe,1006620원,4.9,Western Digital,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,2TB,...,PS5 호환,○,○,방열판 미포함,80mm,22mm,2.38mm,7.5g,2022년,09월
4,//img.danawa.com/prod_img/500000/338/161/img/1...,SK하이닉스 Gold P31 M.2 NVMe,225000원,4.8,SK하이닉스,내장형SSD,M.2 (2280),PCIe3.0x4 (32GT/s),NVMe 1.3,1TB,...,NaN,○,○,방열판 미포함,80mm,22mm,2.3mm,7g,2021년,01월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,//img.danawa.com/prod_img/500000/194/958/img/1...,SOH SATA to USB3.0 어댑터(J-300),9800원,5.0,SOH,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013년,03월
1360,//img.danawa.com/prod_img/500000/117/649/img/1...,라이트컴 COMS A3992 하드케이스,1700원,4.2,라이트컴 COMS,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012년,04월
1361,//img.danawa.com/prod_img/500000/110/649/img/1...,라이트컴 COMS A3993 하드케이스,1700원,4.6,라이트컴 COMS,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012년,04월
1362,//img.danawa.com/prod_img/500000/346/297/img/1...,이지넷유비쿼터스 NEXT-HDD BOX 하드 보호케이스,3660원,4.7,이지넷유비쿼터스,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년,01월


In [3]:
df.to_csv("ssd.csv", index=False, encoding="utf-8-sig")

In [4]:
df.head().to_csv("ssd_sample.csv", index=False, encoding="utf-8-sig")

In [18]:
# 1. 첫 번째 테이블: idx (1000부터 시작)와 a, b 컬럼
ssd_product_df = pd.DataFrame({
    'product_idx': range(1504, 1504 + len(df)),
    'name': df['Name'],
    'brand': df['제조회사'],
    'price': df['Price'],
    'rating': df['Rating'],
    'category': "SSD"
})

# 2. 두 번째 테이블: idx (index+1), table1의 idx를 외래키로, c 컬럼
ssd_img_df = pd.DataFrame({
    'product_image_idx': range(1504, 1504 + len(df)),
    'product_idx': ssd_product_df['product_idx'],  # table1의 idx를 외래키로
    'img_url': df['url']
})

used_columns = ['Name', 'Price', 'url', 'Rating', '제조회사']
# 원래 데이터프레임에서 사용되지 않은 컬럼 선택
remaining_columns = [col for col in df.columns if col not in used_columns]

# table3 생성
ssd_detail_df = pd.DataFrame({
    'ssd_idx': range(1, len(df) + 1),
    'product_idx': ssd_product_df['product_idx']
})
# 나머지 컬럼 추가
for col in remaining_columns:
    ssd_detail_df[col] = df[col]

In [19]:
ssd_img_df.to_csv("ssd_img.csv", index=False, encoding="utf-8-sig")

In [20]:
ssd_product_df.to_csv("ssd_product.csv", index=False, encoding="utf-8-sig")

In [26]:
ssd_detail_df.to_csv("ssd_detail.csv", index=False, encoding="utf-8-sig")

In [24]:
ssd_detail_df.drop(['RAM 탑재', 'RAM 타입', '읽기IOPS', '쓰기IOPS', 'TRIM','GC', 'SLC캐싱','S.M.A.R.T','DEVSLP', 'AES 암호화', '전용 S/W', '게임기 호환', '5년', '제한보증'], axis=1, inplace=True)

In [28]:
ssd_detail_df

,ssd_idx,product_idx,제품분류,폼팩터,인터페이스,프로토콜,용량,메모리 타입,낸드 구조,컨트롤러,...,순차쓰기,MTBF,TBW,NVMe 방열판,가로,세로,두께,무게,등록년,등록월
0,1,1504,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,2TB,TLC(토글),3D낸드,SK하이닉스 Aries,...,"6,500MB/s",150만시간,"1,200TB",방열판 미포함,80mm,22mm,2.38mm,7g,2022년,05월
1,2,1505,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,1TB,TLC(기타),3D낸드,InnoGrit IG5220,...,"4,800MB/s",NaN,700TB,방열판 포함,80mm,22mm,3.4mm,10g,2024년,07월
2,3,1506,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 2.0,2TB,TLC(토글),3D낸드,삼성 Pascal,...,"6,900MB/s",150만시간,"1,200TB",방열판 미포함,80mm,22mm,2.3mm,9g,2022년,11월
3,4,1507,내장형SSD,M.2 (2280),PCIe4.0x4 (64GT/s),NVMe 1.4,2TB,TLC(기타),3D낸드,WD,...,"6,600MB/s",NaN,"1,200TB",방열판 미포함,80mm,22mm,2.38mm,7.5g,2022년,09월
4,5,1508,내장형SSD,M.2 (2280),PCIe3.0x4 (32GT/s),NVMe 1.3,1TB,TLC(토글),3D낸드,SK하이닉스 Cepheus,...,"3,200MB/s",150만시간,750TB,방열판 미포함,80mm,22mm,2.3mm,7g,2021년,01월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,1360,2863,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013년,03월
1360,1361,2864,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012년,04월
1361,1362,2865,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012년,04월
1362,1363,2866,SSD/HDD 주변기기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년,01월
